# Decision tree



## What is a decision tree?

Decision trees ara a non-parametric supervised learning mostly used for classification and regression. This kind ofinferentia model may have problems of overfitting, but it's quite easy to understand and implement.

![alt text](https://scikit-learn.org/stable/_images/sphx_glr_plot_tree_regression_0011.png "Decision Tree")


## 1 - Dependencies

In [2]:
# Matrix operations
import numpy as np
# Tree creation
from sklearn import tree

## Implementation

### 2 - Data Setup

Lets suppouse we have a dataset and we are going to split it into train/test in a .csv document.

The important part of a decission tree is to identify the best branch. To do that, we define a general splitting function:

In [ ]:
# Split a dataset based on an attribute and the value associated
def split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

###  3 - Loss function 
As always, we need to create a function that measures the loss between the current state vector and the new vector updated in that step.

In this special case, the loss function is computed directly on each branch. That function is called gini: (this gini function may need modifications depending on the problem)

In [4]:
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini

SyntaxError: invalid token (<ipython-input-4-3567dfa54382>, line 1)

### 4 - Best Split
Once we have all the splits and their gini, we take the best one:

In [5]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

### 5 - Model
Now that we have a full step set up, we just need to creat the iteration model. 

The thing here is that we can't create an infinite loop. To avoid the problem, we predefine the maximum depth of the tree.

If we want to be picky, we should define the minumum too.

#### 5.1 - End
Before we define a possibly destroying loop for your pc, we should create a function that finish it. In terms of trees, the last node:

In [ ]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

#### 5.2 - Possibilities
We have to define every possibility, combining final nodes or common ones:

In [ ]:
def splits(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

#### 5.3 - Recursion

In [ ]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    splits(root, max_depth, min_size, 1)
    return root

### 6 - Prediction
#### 6.1 
Lets create the prediction function using what we already learn:

In [ ]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

#### 6.2 - Model

In [ ]:
def decision_tree(train, test, max_depth, min_size):
	tree = build_tree(train, max_depth, min_size)
	predictions = list()
	for row in test:
		prediction = predict(tree, row)
		predictions.append(prediction)
	return(predictions)

### 7 - Visualization
Just because this construction is a bit special, we will indicate the way to have a realistic visualization of the tree

In [ ]:
def print_tree(node, depth=0):
	if isinstance(node, dict):
		print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
		print_tree(node['left'], depth+1)
		print_tree(node['right'], depth+1)
	else:
		print('%s[%s]' % ((depth*' ', node)))

### 8 - Test
We will need this hyperparameters:

n_folds

max_depth

min_size
#### 8.1 - Cross validation

In [ ]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

#### 8.2 - Measure

In [ ]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

#### 8.3 - Evaluate

In [ ]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

#### 8.4 Run

In [ ]:
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

### * - Extra
#### Decission Sklearn

In [ ]:
clf = tree.DecisionTreeClassifier(random_state=0, max_depth=2)
clf = clf.fit(data, target)
tree.plot_tree(clf.fit(data, target))

If we want to visualize it even better

In [ ]:
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None, 
                      feature_names=feature_names,  
                      class_names=target_names,  
                      filled=True, rounded=True,  
                      special_characters=True)  
graph = graphviz.Source(dot_data)  
graph 

#### Regression Sklearn

In [1]:
clf = tree.DecisionTreeRegressor()
clf = clf.fit(X, y)
clf.predict(vector)

NameError: name 'tree' is not defined